# Lab 2: Pipeline Interativo de Recomendação de Frequência

Este notebook executa o pipeline de recomendação de frequência de ponta a ponta. As células abaixo irão compilar o código MapReduce em Java, ingerir dados do PostgreSQL com Sqoop, executar o job no Hadoop e verificar os resultados.

## 1. Compilando a Aplicação MapReduce

A primeira etapa é compilar nosso código-fonte Java em um arquivo `.jar` executável. A célula abaixo usa o Maven para isso. Ela navega até o diretório do projeto e executa o `mvn package`. A flag `-q` é para uma saída mais limpa.

In [ ]:
%%bash
cd recommendation-frequency
mvn package -q
echo "JAR compilado com sucesso em: target/recommendation-frequency-1.0-SNAPSHOT.jar"

## 2. Origem dos dados

Execute a célula abaixo para se conectar novamente e fazer uma consulta `SELECT` com os dados inseridos anteriormente.

In [ ]:
import psycopg2
import os

# As credenciais e o host são baseados no arquivo docker-compose.txt
DB_HOST = "localhost" # Nome do serviço no Docker Compose
DB_NAME = "postgres"
DB_USER = "postgres"
DB_USER_PWD = "postgres"

try:
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()
    
    cur.execute("SELECT pet.pet_id, pet.nome, agendamento.data_agendamento FROM agendamento JOIN pet ON agendamento.pet_id = pet.pet_id WHERE agendamento.status = 'Realizado'")
    rows = cur.fetchall()
    
    print("Agendamentos encontrados:")
    for row in rows:
        print(row)
        
except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()

## 3. Ingestão de Dados com Sqoop

Agora, vamos importar os dados de agendamentos do PostgreSQL para o HDFS. O comando `hdfs dfs -rm` é usado para remover o diretório de destino antes da importação, garantindo que possamos executar esta célula várias vezes sem erros.

In [ ]:
%%bash
INPUT_DIR=/petshop/input_agendamentos

# Remove o diretório de entrada se ele já existir
hdfs dfs -test -d $INPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS existente: $INPUT_DIR"
    hdfs dfs -rm -r $INPUT_DIR
fi

echo "Iniciando importação com Sqoop..."
sqoop import \
    --connect jdbc:postgresql://localhost:5432/postgres \
    --username postgres \
    --password postgres \
    --query "SELECT pet.pet_id, pet.nome, agendamento.data_agendamento FROM agendamento JOIN pet ON agendamento.pet_id = pet.pet_id WHERE agendamento.status = 'Realizado' AND \$CONDITIONS" \
    --target-dir $INPUT_DIR \
    --m 1 \
    --split-by pet.pet_id

echo "\nImportação concluída. Verificando os dados no HDFS:"
hdfs dfs -ls $INPUT_DIR
hdfs dfs -cat $INPUT_DIR/part-m-00000 | head -n 5

## 4. Execução do Job MapReduce

Com os dados no HDFS, podemos executar nosso job MapReduce. A célula abaixo submete o `.jar` que compilamos para o Hadoop. O resultado será salvo no diretório `/petshop/output_recomendacoes`.

In [ ]:
%%bash
INPUT_DIR=/petshop/input_agendamentos
OUTPUT_DIR=/petshop/output_recomendacoes
JAR_PATH=recommendation-frequency/target/recommendation-frequency-1.0-SNAPSHOT.jar

# Remove o diretório de saída se ele já existir
hdfs dfs -test -d $OUTPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS de saída existente: $OUTPUT_DIR"
    hdfs dfs -rm -r $OUTPUT_DIR
fi

echo "Executando o job MapReduce..."
hadoop jar $JAR_PATH -D log4j.logger.com.petshop.hadoop.FrequencyMapper=DEBUG $INPUT_DIR $OUTPUT_DIR

echo "\nJob concluído!"

## 5. Verificação do Resultado

Vamos verificar o resultado processado no HDFS. O arquivo `part-r-00000` deve conter o `pet_id`, a `data_sugerida` e a `frequencia_media`.

In [ ]:
%%bash
OUTPUT_DIR=/petshop/output_recomendacoes
echo "Conteúdo do diretório de saída:"
hdfs dfs -ls $OUTPUT_DIR

echo "\nResultado do processamento:"
hdfs dfs -cat $OUTPUT_DIR/part-r-00000

## 6. Carregando e Consultando no Redis (Exemplo)

A etapa final na arquitetura real seria carregar este resultado no Redis para consulta rápida. O serviço `loader-py` faria isso automaticamente. Abaixo, simulamos como os dados seriam armazenados e consultados usando `redis-cli`.

In [ ]:
import redis
redis = redis.Redis(host = 'localhost', port=6379)
redis.ping()

In [ ]:
%%bash
OUTPUT_FILE=/petshop/output_recomendacoes/part-r-00000
REDIS_HOST=localhost

# Verifica se o arquivo de resultado existe
hdfs dfs -test -e $OUTPUT_FILE
if [ $? -ne 0 ]; then
    echo "Erro: Arquivo de resultado não encontrado em $OUTPUT_FILE"
    exit 1
fi

echo "Limpando recomendações antigas no Redis..."
redis-cli -h $REDIS_HOST KEYS "recomendacao:pet:*" | xargs -r redis-cli -h $REDIS_HOST DEL

echo "Carregando novas recomendações do HDFS para o Redis..."
# Lê o arquivo do HDFS e processa linha por linha
hdfs dfs -cat $OUTPUT_FILE | while IFS=$'\t' read -r pet_id values; do
    # Extrai os valores (data e frequência)
    sug_date=$(echo $values | cut -d',' -f1)
    avg_freq=$(echo $values | cut -d',' -f2)
    
    # Monta e executa o comando HSET para o Redis
    echo "Inserindo recomendação para o pet_id: $pet_id"
    redis-cli -h $REDIS_HOST HSET "recomendacao:pet:$pet_id" \
        data_sugerida "$sug_date" \
        frequencia_media_dias "$avg_freq"
done

echo "\nCarga de dados concluída."

echo "Totos os registros inseridos:"
redis-cli -h $REDIS_HOST KEYS "recomendacao:pet:*"